#manipulate_regonline_output

This notebook reads the RegOnline output into a pandas DataFrame and reworks it to have each row contain the attendee, the Doppler Primer Session, the Monday Breakout session, and the Tuesday breakout session in each row.

In [1]:
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib

In [2]:
#%matplotlib inline

### Read the RegOnline output into a pandas DataFrame

In [112]:
df = pd.read_excel('/Users/matt/projects/EPRV/data/AttendeeReportCrop_20150703.xls', encoding='utf-8')

In [113]:
df.columns

Index([u'AgendaItem', u'RegId', u'GroupId', u'FirstName', u'LastName', u'Company'], dtype='object')

In [114]:
df.loc[36:37]

,AgendaItem,RegId,GroupId,FirstName,LastName,Company
36,Doppler Primer: Instrumentation Challenges,78650013,78650013,Rebekah,Dawson,UC Berkeley
37,Doppler Primer: Spot Modeling,78698289,78698289,Fabio,Del Sordo,Yale / NORDITA


### Extract the Sunday Sessions

RegOnline outputs multiple entries for each person, and each entry differs by the `AgendaItem`. `AgendaItem`s exist for all sessions happening on all days. In this section, we extract the sessions happening on Sunday, which are all prefixed by "Doppler Primer: ".

In [115]:
sundf = df[df['AgendaItem'].str.contains('Doppler Primer:')].copy()
len(sundf)

126

Let's create two new columns in our DataFrame: the `Primer`, and the `PrimerID`. The `Primer` column will contain the name of the Doppler Primer session (minus the `Doppler Primer: ` prefix), and the `PrimerID` will be a session identifier that will later be used in plotting.

In [116]:
sundf['PrimerID'] = 0

In [117]:
sundf['Primer'] = [re.search(r'(.*):\s(.*)$', item).group(2) for item in sundf['AgendaItem']]

In [118]:
sundf[['AgendaItem', 'Primer']].head(3)

,AgendaItem,Primer
0,Doppler Primer: Instrumentation Challenges,Instrumentation Challenges
1,Doppler Primer: Doppler code,Doppler code
2,Doppler Primer: Spot Modeling,Spot Modeling


In [119]:
sundf['Primer'].unique()

array([u'Instrumentation Challenges', u'Doppler code', u'Spot Modeling',
       u'Statistical Analysis', u'Not Attending'], dtype=object)

Now loop through the five unique sessions, updating the `PrimerID` column for each participant:

In [120]:
dopID = 0
for agItem in sundf['Primer'].unique():
    sundf.loc[sundf['Primer'] == agItem, 'PrimerID'] = dopID
    dopID += 1

Create an abbreviated code for each session. This will be added to the nametag to spark conversation among participants.

In [121]:
sun_ses = ['IC', 'DC', 'SM', 'SA', 'NA']

A quick preview of the first few rows to see the result:

In [122]:
sundf[['AgendaItem', 'Primer', 'PrimerID']].head(4)

,AgendaItem,Primer,PrimerID
0,Doppler Primer: Instrumentation Challenges,Instrumentation Challenges,0
1,Doppler Primer: Doppler code,Doppler code,1
2,Doppler Primer: Spot Modeling,Spot Modeling,2
3,Doppler Primer: Spot Modeling,Spot Modeling,2


### Extract the Monday Sessions

Now to do the same for the Monday sessions.

In [123]:
mondf = df[df['AgendaItem'].str.contains('Monday Break-out:')].copy()
len(mondf)

135

In [124]:
mondf['MonID'] = 0

mondf['Monday'] = [re.search(r'(.*):\s(.*)$', item).group(2) for item in mondf['AgendaItem']]

mondf['Monday'].unique()

monID = 0
for agItem in mondf['Monday'].unique():
    mondf.loc[mondf['Monday'] == agItem, 'MonID'] = monID
    monID += 1

In [125]:
mondf['Monday'].unique()

array([u'Fiber Optic Scrambling', u'Not attending',
       u'Telluric Contamination', u'Barycentric Corrections',
       u'Frequency Comb Calibrators'], dtype=object)

In [126]:
mon_ses = ['FS', 'NA', 'TC', 'BC', 'FC']

In [127]:
mondf[['AgendaItem', 'Monday', 'MonID']].head(4)

,AgendaItem,Monday,MonID
126,Monday Break-out: Fiber Optic Scrambling,Fiber Optic Scrambling,0
127,Monday Break-out: Not attending,Not attending,1
128,Monday Break-out: Fiber Optic Scrambling,Fiber Optic Scrambling,0
129,Monday Break-out: Telluric Contamination,Telluric Contamination,2


### Extract Tuesday Sessions

In [128]:
tuedf = df[df['AgendaItem'].str.contains('Tuesday Break-out:')].copy()
len(tuedf)

135

In [129]:
tuedf['TueID'] = 0

tuedf['Tuesday'] = [re.search(r'(.*):\s(.*)$', item).group(2) for item in tuedf['AgendaItem']]

tuedf['Tuesday'].unique()

tuesID = 0
for agItem in tuedf['Tuesday'].unique():
    tuedf.loc[tuedf['Tuesday'] == agItem, 'TueID'] = tuesID
    tuesID += 1

In [130]:
tuedf['Tuesday'].unique()

array([u'Statistical techniques', u'Detection Threshold Criteria',
       u'Line Bisectors and decorrelatio', u'Photospheric Signals',
       u'Not attending'], dtype=object)

In [131]:
tue_ses = ['ST', 'DC', 'LB', 'PS', 'NA']

In [132]:
tuedf[['AgendaItem', 'Tuesday', 'TueID']].head(4)

,AgendaItem,Tuesday,TueID
261,Tuesday Break-out: Statistical techniques,Statistical techniques,0
262,Tuesday Break-out: Statistical techniques,Statistical techniques,0
263,Tuesday Break-out: Detection Threshold Criteria,Detection Threshold Criteria,1
264,Tuesday Break-out: Detection Threshold Criteria,Detection Threshold Criteria,1


### Combine the DataFrames

We only need to join on one field. However, pandas does something weird, where it creates multiple `GroupId_x` columns when joining multiple times. The simple solution is just to join on multiple columns since we know they're all consistent.

In [133]:
fulldf = df[['RegId', 'GroupId', 'FirstName', 'LastName', 'Company']]

In [134]:
print(len(fulldf))
fulldf = fulldf.drop_duplicates()
print(len(fulldf))
print(len(sundf))
print(len(mondf))
print(len(tuedf))

396
137
126
135
135


In [135]:
fulldf.columns

Index([u'RegId', u'GroupId', u'FirstName', u'LastName', u'Company'], dtype='object')

In [136]:
sundf.columns

Index([u'AgendaItem', u'RegId', u'GroupId', u'FirstName', u'LastName', u'Company', u'PrimerID', u'Primer'], dtype='object')

In [137]:
newdf = pd.merge(fulldf, sundf, on=['RegId', 'GroupId', 'FirstName', 'LastName', 'Company'], how='left')
print(len(newdf))

newdf = pd.merge(newdf, mondf, on=['RegId', 'GroupId', 'FirstName', 'LastName', 'Company'], how='left')
print(len(newdf))

newdf = pd.merge(newdf, tuedf, on=['RegId', 'GroupId', 'FirstName', 'LastName', 'Company'], how='left')
print(len(newdf))

137
137
137


In [138]:
newdf.head(5)

,RegId,GroupId,FirstName,LastName,Company,AgendaItem_x,PrimerID,Primer,AgendaItem_y,MonID,Monday,AgendaItem,TueID,Tuesday
0,79762584,79762584,Arthur,Adams,Yale University,Doppler Primer: Instrumentation Challenges,0,Instrumentation Challenges,Monday Break-out: Fiber Optic Scrambling,0,Fiber Optic Scrambling,Tuesday Break-out: Statistical techniques,0,Statistical techniques
1,79809320,79809320,Guillem,Anglada-Escude,Queen Mary University of London/University of ...,Doppler Primer: Doppler code,1,Doppler code,Monday Break-out: Fiber Optic Scrambling,0,Fiber Optic Scrambling,Tuesday Break-out: Detection Threshold Criteria,1,Detection Threshold Criteria
2,80253872,80253872,Ruth,Angus,University of Oxford/Harvard-Smithsonian Cente...,Doppler Primer: Spot Modeling,2,Spot Modeling,Monday Break-out: Telluric Contamination,2,Telluric Contamination,Tuesday Break-out: Detection Threshold Criteria,1,Detection Threshold Criteria
3,78665380,78665380,Pamela,Arriagada,"DTM, CIW",Doppler Primer: Spot Modeling,2,Spot Modeling,Monday Break-out: Barycentric Corrections,3,Barycentric Corrections,Tuesday Break-out: Line Bisectors and decorrel...,2,Line Bisectors and decorrelatio
4,78665381,78665381,Mariona,Badenas,Yale University,Doppler Primer: Statistical Analysis,3,Statistical Analysis,Monday Break-out: Barycentric Corrections,3,Barycentric Corrections,Tuesday Break-out: Line Bisectors and decorrel...,2,Line Bisectors and decorrelatio


In [139]:
newdf.columns

Index([u'RegId', u'GroupId', u'FirstName', u'LastName', u'Company', u'AgendaItem_x', u'PrimerID', u'Primer', u'AgendaItem_y', u'MonID', u'Monday', u'AgendaItem', u'TueID', u'Tuesday'], dtype='object')

Now create a new DataFrame that is a subset of the `newdf` with only the columns of interest. Also, make sure the DataFrame is sorted by lastname, the index is reset, and it's a copy of `newdf` instead of a pointer to `newdf`.

In [140]:
finaldf = newdf[['FirstName', 'LastName', 'Company', 'Primer', 'PrimerID', 'Monday', 'MonID', 'Tuesday', 'TueID']].sort('LastName').reset_index().copy()

In [141]:
finaldf.head(5)

,index,FirstName,LastName,Company,Primer,PrimerID,Monday,MonID,Tuesday,TueID
0,0,Arthur,Adams,Yale University,Instrumentation Challenges,0,Fiber Optic Scrambling,0,Statistical techniques,0
1,126,Rachel,Akeson,NASA Exoplanet Science Institute,NaN,NaN,Not attending,1,Statistical techniques,0
2,1,Guillem,Anglada-Escude,Queen Mary University of London/University of ...,Doppler code,1,Fiber Optic Scrambling,0,Detection Threshold Criteria,1
3,2,Ruth,Angus,University of Oxford/Harvard-Smithsonian Cente...,Spot Modeling,2,Telluric Contamination,2,Detection Threshold Criteria,1
4,3,Pamela,Arriagada,"DTM, CIW",Spot Modeling,2,Barycentric Corrections,3,Line Bisectors and decorrelatio,2


In [142]:
len(finaldf)

137

In [143]:
finaldf.columns

Index([u'index', u'FirstName', u'LastName', u'Company', u'Primer', u'PrimerID', u'Monday', u'MonID', u'Tuesday', u'TueID'], dtype='object')

Now replace all empty cells for "Company" to a very general location:

In [144]:
finaldf.Company = ['Earth' if pd.isnull(company_el) else company_el for company_el in finaldf.Company]

Replace NaNs for PrimerID with the "Not Attending" ID:

In [145]:
finaldf.PrimerID = [4 if pd.isnull(primerid_el) else primerid_el for primerid_el in finaldf.PrimerID]

Check for NaNs in the Monday ID:

In [146]:
len(finaldf[pd.isnull(finaldf['MonID'])])

2

Replace NaNs for the MonID with the "Not Attending" ID:

In [147]:
finaldf.MonID = [4 if pd.isnull(monid_el) else monid_el for monid_el in finaldf.MonID]

In [148]:
len(finaldf[pd.isnull(finaldf['MonID'])])

0

Replace NaNs for the TueID with the "Not Attending" ID:

In [149]:
len(finaldf[pd.isnull(finaldf['TueID'])])

2

In [150]:
finaldf.TueID = [4 if pd.isnull(tueid_el) else tueid_el for tueid_el in finaldf.TueID]

In [151]:
len(finaldf[pd.isnull(finaldf['TueID'])])

0

Test out the wrap-around text for institute for participants that have long institution names. This regular expression will look for institutions (or Companies, as RegOnline refers to them), and find items that have a '/', and if no '/', either a '-', ',', or 'at' in the text. If so, add a newline character to make the text wrap around to the next line.

We'll first test the output on a participant's institution that contains both a '/' and a '-':

In [152]:
p = re.compile ('(/|^(?!.*/).*-|^(?!.*/).*,|^(?!.*/).*\sat\s)')
p.subn(r'\1\n', finaldf.loc[2].Company)[0]

u'Queen Mary University of London/\nUniversity of Herfordshire'

And test a cell that is long, contains `at`, but `at` is part of a longer word:

In [153]:
p.subn(r'\1\n', finaldf.loc[53].Company)[0]

u'The Ohio State University'

And a quick test on a few more institutions:

In [154]:
[p.sub(r'\1\n', company_el) if len(company_el) > 30 else company_el for company_el in finaldf.head(5).Company.values]

[u'Yale University',
 u'NASA Exoplanet Science Institute',
 u'Queen Mary University of London/\nUniversity of Herfordshire',
 u'University of Oxford/\nHarvard-Smithsonian Center for Astrophysics',
 u'DTM, CIW']

Now update the full `Company` column of the DataFrame:

In [155]:
finaldf.Company = [p.sub(r'\1\n', company_el) if len(company_el) > 30 else company_el for company_el in finaldf.Company.values]

## Plot Labels

Now that we have our DataFrame cleaned up the way we want it we can print the data to the Avery 5392 format. This format contains 6 4"x3" nametags per sheet.

In [156]:
png = mpimg.imread('/Users/matt/projects/EPRV/images/NameTag2.png')

In [157]:
png.shape

(900, 1200, 4)

In [158]:
import matplotlib.font_manager as mfm
fontpaths = fontpaths=['/System/Library/Fonts/',
                       '/Library/Fonts',
                       '/Library/Fonts/Microsoft',
                       '/usr/X11/lib/X11/fonts',
                       '/opt/X11/share/fonts',
                       '/Users/matt/Library/Fonts']

blaa = mfm.findSystemFonts(fontpaths=fontpaths)

In [159]:
colors = ['#FFE2A9', '#4BA4D8', '#768085', '#BF5338', '#335B8F']
colors2 = ['#335B8F', '#BF5338', '#768085',  '#4BA4D8', '#FFE2A9']
colors3 = ['#4BA4D8', '#FFE2A9', '#BF5338', '#768085', '#335B8F']

circ_ypos = 775
name_dict = {'family': 'YaleNew-Roman',
             'color': '#D6E8E1',
             'weight': 'bold',
             'size': 28
             }

company_dict = {'family': 'YaleNew-Roman',
                'color': '#D6E8E1',
                'weight': 'bold',
                'size': 16
                }

circle_dict = {'family': 'YaleNew-Roman',
               'color': '#1D2523',
               'weight': 'normal',
               'size': 20
               }


def change_name_size(name, name_dict):
    if len(name) < 16:
        name_dict['size'] = 28
    elif ((len(name) >= 16) and (len(name) < 19)):
        name_dict['size'] = 24
    elif ((len(name) >= 19) and (len(name) < 24)):
        name_dict['size'] = 20
    elif ((len(name) >= 24) and (len(name) < 30)):
        name_dict['size'] = 17
    else:
        name_dict['size'] = 16
    return name_dict
        

def change_company_size(company, company_dict):
    newlines = len(re.findall(r'\n', finaldf.loc[0].Company))
    if newlines == 0:
        if len(company) < 15:
            company_dict['size'] = 18
        elif ((len(company) >= 15) and (len(company) < 30)):
            company_dict['size'] = 14
        elif ((len(company) >= 30) and (len(company) < 40)):
            company_dict['size'] = 12
        elif ((len(company) >= 40) and (len(company) < 50)):
            company_dict['size'] = 10
        else:
            company_dict['size'] = 8
    else:
        if len(company) < 15:
            company_dict['size'] = 18
        elif ((len(company) >= 15) and (len(company) < 40)):
            company_dict['size'] = 14
        elif ((len(company) >= 40) and (len(company) < 50)):
            company_dict['size'] = 12
        else:
            company_dict['size'] = 10
    return company_dict
    

# The HP Color LaserJet CP4020 offsets things by 1/16th of an inch left-to-right.
# This fudge factor should fix that:
hrz_fdg = 1. / 16./ 8.5
leftarr = np.array([0.0294, 0.5, 0.0294, 0.5, 0.0294, 0.5]) + hrz_fdg
bottomarr = [0.091, 0.091, 0.364,  0.364, 0.637, 0.637]
width = 0.4706
height = 0.273

# loop through the total number of pages:
for page in range(int(np.ceil((len(finaldf))/6.))):
    print('Now on page: {}'.format(page))
    fig = plt.figure(figsize=(8.5, 11))
    for indx in range(6):
        # add an if statement to handle the last page if there are less than
        # six participants remaining:
        if ((page*6 + indx) < len(finaldf)):
            rect = [leftarr[indx], bottomarr[indx], width, height]
            ax = fig.add_axes(rect)
            ax.imshow(png)
            ax.get_xaxis().set_visible(False)
            ax.get_yaxis().set_visible(False)

            print(u'Now making name tag for: {} {}'.format(finaldf.loc[page*6 + indx].FirstName, finaldf.loc[page*6 + indx].LastName))
            
            #add name text:
            name = finaldf.loc[page*6 + indx].FirstName + ' ' + finaldf.loc[page*6 + indx].LastName 
            this_name_dict = change_name_size(name, name_dict)
            ax.text(600, 500, name, fontdict=this_name_dict, horizontalalignment='center')

            #add company text:
            company = finaldf.loc[page*6 + indx].Company
            this_co_dict = change_company_size(company, company_dict)
            ax.text(600, 625, company, fontdict=this_co_dict, horizontalalignment='center')

            #add circles for sessions:
            circ1 = plt.Circle((750, circ_ypos), 70, color=colors[int(finaldf.loc[page*6 + indx].PrimerID)])
            fig.gca().add_artist(circ1)
            ax.text(750, circ_ypos + 27.5, sun_ses[int(finaldf.loc[page*6 + indx].PrimerID)], fontdict=circle_dict, horizontalalignment='center')

            circ2 = plt.Circle((925, circ_ypos), 70, color=colors2[int(finaldf.loc[page*6 + indx].MonID)])
            fig.gca().add_artist(circ2)
            ax.text(925, circ_ypos + 27.5, mon_ses[int(finaldf.loc[page*6 + indx].MonID)], fontdict=circle_dict, horizontalalignment='center')

            circ3 = plt.Circle((1100, circ_ypos), 70, color=colors3[int(finaldf.loc[page*6 + indx].TueID)])
            fig.gca().add_artist(circ3)
            ax.text(1100, circ_ypos + 27.5, tue_ses[int(finaldf.loc[page*6 + indx].TueID)], fontdict=circle_dict, horizontalalignment='center')

    plt.savefig('../nametags/nameTags_bold_p'+str(page)+'.png', dpi=300)


Now on page: 0
Now making name tag for: Arthur Adams
Now making name tag for: Rachel Akeson
Now making name tag for: Guillem Anglada-Escude
Now making name tag for: Ruth Angus
Now making name tag for: Pamela Arriagada
Now making name tag for: Mariona Badenas
Now on page: 1
Now making name tag for: Roman Baluev
Now making name tag for: Fabienne Bastien
Now making name tag for: Ozgur Basturk
Now making name tag for: Sarbani Basu
Now making name tag for: Florian Bauer
Now making name tag for: Jacob Bean
Now on page: 2
Now making name tag for: Thomas Beatty
Now making name tag for: Eric Bechter
Now making name tag for: Megan Bedell
Now making name tag for: Sagi Ben-Ami
Now making name tag for: Cullen Blake
Now making name tag for: Andreas Boesch
Now on page: 3
Now making name tag for: Adam Bolton
Now making name tag for: Francesco Borsa
Now making name tag for: Francois Bouchy
Now making name tag for: Brendan Bowler
Now making name tag for: Tabetha Boyajian
Now making name tag for: John Br

In [52]:
finaldf.columns

Index([u'index', u'FirstName', u'LastName', u'Company', u'Primer', u'PrimerID', u'Monday', u'MonID', u'Tuesday', u'TueID'], dtype='object')

In [54]:
finaldf.FirstName.values

array([u'Arthur', u'Rachel', u'Guillem', u'Ruth', u'Pamela', u'Mariona',
       u'Roman', u'Fabienne', u'Ozgur', u'Sarbani', u'Florian', u'Jacob',
       u'Thomas', u'Eric', u'Megan', u'Sagi', u'Cullen', u'Andreas',
       u'Adam', u'Francesco', u'Francois', u'Brendan', u'Tabetha', u'John',
       u'Lars A.', u'Jennifer', u'Richard', u'Maria Federica', u'Abhijit',
       u'Jessi', u'William', u'Uriel', u'Matthew', u'Justin', u'Mario',
       u'Allen', u'Rebekah', u'Fabio', u'Rodrigo', u'Scott', u'Courtney',
       u'Xavier', u'Jason', u'Michael', u'Jo\xe3o', u'Tobias', u'Pedro',
       u'Debra', u'Eric', u'Daniel', u'BJ', u'Gabor', u'Peter', u'Bernard',
       u'Matteo', u'Paolo', u'Matt', u'Steve', u'Steve', u'Erica',
       u'Frank', u'Guillaume', u'Sam', u'Artie', u'Raphaelle',
       u'Guillaume', u'Enrique', u'David', u'Joshua', u'Andrew', u'John',
       u'Paul', u'Colby', u'Marco', u'Antonino Francesco', u'David',
       u'Gregory', u'Christophe', u'Zaira', u'Bo', u'Gregory', u'

In [62]:
finaldf.LastName.values

array([u'Adams', u'Akeson', u'Anglada-Escude', u'Angus', u'Arriagada',
       u'Badenas', u'Baluev', u'Bastien', u'Basturk', u'Basu', u'Bauer',
       u'Bean', u'Beatty', u'Bechter', u'Bedell', u'Ben-Ami', u'Blake',
       u'Boesch', u'Bolton', u'Borsa', u'Bouchy', u'Bowler', u'Boyajian',
       u'Brewer', u'Buchhave', u'Burt', u'Capps', u'Cersullo',
       u'Chakraborty', u'Cisewski', u'Cochran', u'Conod', u'Cornachione',
       u'Crepp', u'Damasso', u'Davis', u'Dawson', u'Del Sordo', u'Diaz',
       u'Diddams', u'Dressing', u'Dumusque', u'Eastman', u'Endl', u'Faria',
       u'Feger', u'Figueira', u'Fischer', u'Ford', u'Foreman-Mackey',
       u'Fulton', u'Furesz', u'Gao', u'Gaudi', u'Genoni', u'Giacobbe',
       u'Giguere', u'Girvin', u'Girvin', u'Gonzales', u'Grundahl',
       u'HEBRARD', u'Halverson', u'Hatzes', u'Haywood', u'Hebrard',
       u'Herrero', u'Hogg', u'Hopgood', u'Howard', u'Johnson', u'Jorden',
       u'Jurgenson', u'Landoni', u'Lanza', u'Latham', u'Laughlin',
       

In [59]:
hrz_fdg = 1. / 16./ 8.5
leftarr = np.array([0.0294, 0.5, 0.0294, 0.5, 0.0294, 0.5])


In [60]:
leftarr + hrz_fdg

array([ 0.03675294,  0.50735294,  0.03675294,  0.50735294,  0.03675294,
        0.50735294])